# 1-Generate Predictions using LangChain

- **Goal:** Prediction Recognition

- **Purpose:** To implement step 1 with sub steps of prediction recognition pipeline. See steps
    1. Generate predictions
        1. Create several prediction prompts templates
        2. Utilize open-source LLMs to generate predictions

- **Misc:**
    - `%store`: Cell magic will store the variable of interest so we can load in another notebook

In [1]:
import os, sys

import pandas as pd

from tqdm import tqdm
from langchain_core.prompts import PipelinePromptTemplate, PromptTemplate

# Get the current working directory of the notebook
notebook_dir = os.getcwd()
# Add the parent directory to the system path
sys.path.append(os.path.join(notebook_dir, '../'))

from log_files import LogData
from data_processing import DataProcessing
from text_generation_models import TextGenerationModelFactory

In [2]:
# pd.set_option('max_colwidth', 800)

groq_cloud_tgmf = TextGenerationModelFactory()
llama_versatile_generation_model = groq_cloud_tgmf.create_instance(model_name='llama-3.3-70b-versatile')
llama_instant_generation_model = groq_cloud_tgmf.create_instance('llama-3.1-8b-instant')
llama_70b_8192_generation_model = groq_cloud_tgmf.create_instance('llama3-70b-8192')
llama_8b_8192_generation_model = groq_cloud_tgmf.create_instance('llama3-8b-8192')

navi_gator_tgmf = TextGenerationModelFactory()
gpt_35_turbo_generation_model = navi_gator_tgmf.create_instance('gpt-3.5-turbo')
gpt_4_turbo_generation_model = navi_gator_tgmf.create_instance('gpt-4-turbo')
mixtral_87b_instruct_generation_model = navi_gator_tgmf.create_instance('mixtral-8x7b-instruct') 

## LangChain Templates for Domain Predictions

In [3]:
full_prediction_template = """{prediction_properties}

{prediction_requirements}

{prediction_templates}

{prediction_examples}
"""

full_prediction_prompt = PromptTemplate.from_template(full_prediction_template)

In [4]:
prediction_properties_template = """A prediction <p> = (<p_s>, <p_t>, <p_d>, <p_a>), where it consists of the following four properties:

    1. <p_s>, any source entity in the {prediction_domain} domain.
        - Can be a person (with a name) or a {prediction_domain} person such as a {prediction_domain} reporter, {prediction_domain} analyst, {prediction_domain} expert, {prediction_domain} top executive, {prediction_domain} senior level person, etc).
        - Can only be an organization that is associated with the {prediction_domain} prediction.
    2. <p_t>, any target entity in the {prediction_domain} domain.
	    - Can be a person (with a name) or a {prediction_domain} person such as a {prediction_domain} reporter, {prediction_domain} analyst, {prediction_domain} expert, {prediction_domain} top executive, {prediction_domain} senior level person, etc).
        - Can only be an organization that is associated with the {prediction_domain} prediction.
    3. <p_d>, date range when <p> is expected to come to fruition.
        - Forecast can range from a second to anytime in the future.
        - Answers the questions: "How far to go out from today?" or "Where to stop?".
    4. <p_a>, {prediction_domain} prediction attribute.
        - Characteristics of a domain-specific attributes such as various quantifiable metrics relevant to the {prediction_domain} domain.
        - Some examples are {prediction_domain_attribute}.  
"""
prediction_properties_prompt = PromptTemplate.from_template(prediction_properties_template)

In [5]:
prediction_requirements_template = """{prediction_domain} requirements to use for each prediction:

    - Should be based on real-world {prediction_domain} data and not hallucinate.
    - Only a simple sentence (prediction) (and NOT compounding using "and" or "or").
    - Should diversify all four properties of the prediction (<p>) as in change and not use same for <p_s>, <p_t>, <p_d>, <p_a>.
    - Should use synonyms to predict such as forecasts, speculates, foresee, envision, etc., and not use any of them more than ten times.
    - The prediction should be unique and not repeated.
    - Do not number the predictions.
    - Do not say, "As the {prediction_domain}, I will generate company-based {prediction_domain} predictions using the provided templates." or anything similar.
    - Use the five different templates and examples provided.
    - Change how the current date (<p_d>) written in the prediction with examples of (1) Wednesday, August 21, 2024; (2) Wed, August 21, 2024; (3) 08/21/2024; (4) 08/21/2024; (5) 21/08/2024; (6) 21 August 2024; (7) 2024/08/21; (8) 2024-08-21; (9) August 21, 2024; (10) Aug 21, 2024; (11) 21 August 2024, (12) 21 Aug 2024, Q3 of 2027, 2029 of Q3, etc (with removing day of week).
    {domain_requirements}
    - Do not say, "Here are {predictions_N} unique {prediction_domain} predictions based on the provided templates and examples:" in the prompt.
    - Do not use any of the examples in the prompt.
    - In front of every prodiction, put the template number in the format of "T1:", "T2:", etc. and do not number them like "1.", "2.", etc.
    - Do not put template number on line by itself. Always pair with a prediction.
    - Disregard brackets: "<>"
    - Should never say "Here are {predictions_N} unique {prediction_domain} predictions based on the provided templates and examples:" or "Note: I've made sure to follow the guidelines and templates provided, and generated unique predictions that meet the requirements."
    - Do not use person name of entity name more than once as in don't use name Joe as both the <p_s> and <p_t>, unless like Mr. Sach and Goldman Sach or Mr. Sam Walton and Sam's Club, etc.
    - Should variate the slope of rise/increase/as much as, fall/decrease/as little as, change, stay stable, high/low chance/probability/degree of, etc.
    - Should variate the prediction verbs such as will, would, be going to, should, etc.
"""
prediction_requirements_prompt = PromptTemplate.from_template(prediction_requirements_template)

In [6]:
prediction_templates_template = """Here are some {prediction_domain} templates:

- {prediction_domain} template 1: <p_s> forecasts that the <p_a> at <p_t> will likely decrease in <p_d>.

"""
prediction_templates_prompt = PromptTemplate.from_template(prediction_templates_template)

In [7]:
prediction_examples_template = """Here are some examples of {prediction_domain} predictions:

{domain_examples}

With the above, generate a unique set of {predictions_N} predictions. Think from the perspective of an {prediction_domain} analyst, expert, top executive, or senior level person."""
prediction_examples_prompt = PromptTemplate.from_template(prediction_examples_template)

In [8]:
prediction_input_prompts = [
    ("prediction_properties", prediction_properties_prompt),
    ("prediction_requirements", prediction_requirements_prompt),
    ("prediction_templates", prediction_templates_prompt),
    ("prediction_examples", prediction_examples_prompt),
]

pipeline_prompt = PipelinePromptTemplate(
    final_prompt=full_prediction_prompt, pipeline_prompts=prediction_input_prompts
)

/var/folders/78/9z0b45fx1xqbwxh8vk97lcfh0000gn/T/ipykernel_72555/3566428802.py:8: LangChainDeprecationWarning: This class is deprecated. Please see the docstring below or at the link for a replacement option: https://python.langchain.com/api_reference/core/prompts/langchain_core.prompts.pipeline.PipelinePromptTemplate.html
  pipeline_prompt = PipelinePromptTemplate(


## Generate Domain Predictions

In [9]:
predictions_N = 2

### Generate Financial Predictions

In [10]:
financial_attributes = """stock price, net profit, revenue, operating cash flow, research and development expenses, operating income, gross profit."""
financial_requirements = """- Should be based on real-world financial earnings reports.
    - Suppose the time when $p$ was made is during any earning season.
    - Include stocks from all sectors such as consumer staples, energy, finance, health care, industrials, materials, media, real estate, retail, technology, utilities, defense, etc.
    - Include the US Dollar sign ($) before or USD after the amount of the financial attribute."""

financial_examples = """
    1. Detravious, an investor forecasts that the stock price at Apple will likely decrease in 2025 Q1.
    2. Ava Lee predicts that the operating cash flow at ExxonMobil should decrease in 03/21/2025 to 08/21/2025.
 """

In [11]:
financial_input_dict = {
    "prediction_domain": "financial",
    "prediction_domain_attribute": financial_attributes,
    "domain_requirements": financial_requirements,
    "domain_examples": financial_examples,
    "predictions_N": predictions_N
}
financial_prompt_output = pipeline_prompt.format(**financial_input_dict)
print(financial_prompt_output)


A prediction <p> = (<p_s>, <p_t>, <p_d>, <p_a>), where it consists of the following four properties:

    1. <p_s>, any source entity in the financial domain.
        - Can be a person (with a name) or a financial person such as a financial reporter, financial analyst, financial expert, financial top executive, financial senior level person, etc).
        - Can only be an organization that is associated with the financial prediction.
    2. <p_t>, any target entity in the financial domain.
	    - Can be a person (with a name) or a financial person such as a financial reporter, financial analyst, financial expert, financial top executive, financial senior level person, etc).
        - Can only be an organization that is associated with the financial prediction.
    3. <p_d>, date range when <p> is expected to come to fruition.
        - Forecast can range from a second to anytime in the future.
        - Answers the questions: "How far to go out from today?" or "Where to stop?".
    4. 

In [12]:
N_batches = 1
# text_generation_models = [llama_instant_generation_model]
# text_generation_models = [llama_versatile_generation_model]
text_generation_models = [llama_versatile_generation_model, llama_instant_generation_model, llama_70b_8192_generation_model, 
                          llama_8b_8192_generation_model, gpt_35_turbo_generation_model, gpt_4_turbo_generation_model, 
                          mixtral_87b_instruct_generation_model]

# text_generation_models = [llama_versatile_generation_model, llama_instant_generation_model, llama_70b_8192_generation_model, 
#                           llama_8b_8192_generation_model, gpt_35_turbo_generation_model]

In [13]:
prediction_domains = ["finance"]
prediction_prompt_outputs = {
    "finance": financial_prompt_output,
}
prediction_label = 1

batched_predictions_df = groq_cloud_tgmf.batch_generate_predictions(N_batches=N_batches, 
                                text_generation_models=text_generation_models, 
                                domains=prediction_domains,
                                prompt_outputs=prediction_prompt_outputs,
                                sentence_label=prediction_label)

  0%|          | 0/1 [00:00<?, ?it/s]

finance --- <text_generation_models.LlamaVersatileTextGenerationModel object at 0x1618162a0>
finance --- <text_generation_models.LlamaInstantTextGenerationModel object at 0x16233f9e0>
finance --- <text_generation_models.Llama70B8192TextGenerationModel object at 0x162368cb0>
finance --- <text_generation_models.Llama8B8192TextGenerationModel object at 0x162369f10>
finance --- <text_generation_models.Gpt35TurboTextGenerationModel object at 0x16236b230>
finance --- <text_generation_models.Gpt4TurboTextGenerationModel object at 0x16237cce0>
finance --- <text_generation_models.Mixtral87BInstructTextGenerationModel object at 0x16237e9c0>


100%|██████████| 1/1 [00:07<00:00,  7.25s/it]

In [14]:
batched_predictions_df

[                                       Base Sentence  Sentence Label  \
 0  T1: Goldman Sachs forecasts that the revenue a...               1   
 1  T2: JPMorgan Chase predicts that the net profi...               1   
 
                 Model Name   Domain  Batch Index  
 0  llama-3.3-70b-versatile  finance            0  
 1  llama-3.3-70b-versatile  finance            0  ,
                                        Base Sentence  Sentence Label  \
 0  T1: Mr. Thompson, a financial expert, forecast...               1   
 1  T2: Samantha, a financial analyst, predicts th...               1   
 
              Model Name   Domain  Batch Index  
 0  llama-3.1-8b-instant  finance            0  
 1  llama-3.1-8b-instant  finance            0  ,
                                        Base Sentence  Sentence Label  \
 0         Here are two unique financial predictions:               1   
 1  T1: Samantha Jenkins, a financial expert, fore...               1   
 2  T5: Goldman Sachs envisions th

In [15]:
pd.set_option('max_colwidth', 800)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
batched_predictions_df

[                                                                                                            Base Sentence  \
 0                           T1: Goldman Sachs forecasts that the revenue at Microsoft will likely increase in Q4 of 2024.   
 1  T2: JPMorgan Chase predicts that the net profit at Amazon should decrease in the period from 2024-08-21 to 2025-02-20.   
 
    Sentence Label               Model Name   Domain  Batch Index  
 0               1  llama-3.3-70b-versatile  finance            0  
 1               1  llama-3.3-70b-versatile  finance            0  ,
                                                                                                                         Base Sentence  \
 0                   T1: Mr. Thompson, a financial expert, forecasts that the net profit at Microsoft will likely decrease in 2027 Q3.   
 1  T2: Samantha, a financial analyst, predicts that the research and development expenses at Tesla should increase in 21 August 2024.   
 

In [16]:
predictions_df = DataProcessing.concat_dfs(batched_predictions_df)
predictions_df

,Base Sentence,Sentence Label,Model Name,Domain,Batch Index
0,T1: Goldman Sachs forecasts that the revenue at Microsoft will likely increase in Q4 of 2024.,1,llama-3.3-70b-versatile,finance,0
1,T2: JPMorgan Chase predicts that the net profit at Amazon should decrease in the period from 2024-08-21 to 2025-02-20.,1,llama-3.3-70b-versatile,finance,0
2,"T1: Mr. Thompson, a financial expert, forecasts that the net profit at Microsoft will likely decrease in 2027 Q3.",1,llama-3.1-8b-instant,finance,0
3,"T2: Samantha, a financial analyst, predicts that the research and development expenses at Tesla should increase in 21 August 2024.",1,llama-3.1-8b-instant,finance,0
4,Here are two unique financial predictions:,1,llama3-70b-8192,finance,0
5,"T1: Samantha Jenkins, a financial expert, forecasts that the revenue at Tesla will likely increase in Q2 of 2024.",1,llama3-70b-8192,finance,0
6,T5: Goldman Sachs envisions that the net profit at Coca-Cola will stay stable from 2024-08-15 to 2024-11-15.,1,llama3-70b-8192,finance,0
7,Here are two unique financial predictions based on the provided templates and examples:,1,llama3-8b-8192,finance,0
8,T1: Morgan Stanley forecasts that the revenue at Amazon will likely increase in Q3 of 2027.,1,llama3-8b-8192,finance,0
9,T2: Dr. Sophia Patel speculates that the net profit at Johnson & Johnson will likely stay stable from 2024-08-21 to 2024-12-31.,1,llama3-8b-8192,finance,0


In [ ]:
log_directory = '../data/prediction_logs/'
file_name = 'test.log'

In [ ]:
logger = LogData(log_directory, file_name)
logger

In [ ]:
csv_output_path = os.path.join(log_directory, 'from_dataframe.csv')
logger.dataframe_to_csv(predictions_df, csv_output_path)

In [ ]:
csv_input_path = os.path.join(log_directory, 'from_dataframe.csv')
file_name = 'from_csv.log'
logger.csv_to_log(csv_input_path, file_name)

In [ ]:
csv_output_from_log_path = os.path.join(log_directory, 'from_log.csv')
ignore_patterns = ['INFO', 'DEBUG', 'ERROR', 'WARNING', 'CSV Row: '] # Example patterns to ignore

logger.log_to_csv(file_name, csv_output_from_log_path, ignore_patterns)

In [ ]:
csv_input_from_log_path = os.path.join(log_directory, 'from_dataframe.csv')
df_from_log_csv = logger.csv_to_dataframe(csv_input_from_log_path)

if df_from_log_csv is not None:
    print("\nDataFrame created from log CSV:")
    print(df_from_log_csv)
else:
    print("\nFailed to create DataFrame from log CSV. Check the log file for errors.")

In [ ]:
df_from_log_csv